In [2]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [3]:
!pip install SQLAlchemy==1.4

     |████████████████████████████████| 1.5 MB 1.8 MB/s eta 0:00:01
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 1.4.20
    Uninstalling SQLAlchemy-1.4.20:
      Successfully uninstalled SQLAlchemy-1.4.20


# Проект по SQL 

Цели исследования: проанализировать базу данных.

Описание данных: в базе данных хранятся таблицы с информацией о книгах, издательствах, авторах, а также пользовательские обзоры книг. 

Задачи исследования: 
* Посчитать, сколько книг вышло после 1 января 2000 года;
* Для каждой книги посчитать количество обзоров и среднюю оценку;
* Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
* Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
* Посчитайть среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [4]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 
query = 'select 1'
pd.io.sql.read_sql(query, con = engine)

,?column?
0,1


In [5]:
#выведем первые строки таблицы books
books = 'SELECT * FROM books LIMIT 1'
pd.io.sql.read_sql(books, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93


In [6]:
#выведем первые строки таблицы authors
authors = 'SELECT * FROM authors LIMIT 1'
pd.io.sql.read_sql(authors, con = engine)

,author_id,author
0,1,A.S. Byatt


In [7]:
#выведем первые строки таблицы publishers
publishers = 'SELECT * FROM publishers LIMIT 1'
pd.io.sql.read_sql(publishers, con = engine)

,publisher_id,publisher
0,1,Ace


In [8]:
#выведем первые строки таблицы ratings
ratings = 'SELECT * FROM ratings LIMIT 1'
pd.io.sql.read_sql(ratings, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4


In [9]:
#выведем первые строки таблицы reviews
reviews = 'SELECT * FROM reviews LIMIT 1'
pd.io.sql.read_sql(reviews, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...


In [10]:
#Так как главной сущностью схемы является таблица books - проверим распределение данных в части столбцов на уникальность
unique_check = 'SELECT count(*), count(distinct book_id) as book_id, count(distinct author_id) as author_id,count(distinct title) as title from books'
pd.io.sql.read_sql(unique_check, con = engine)

,count,book_id,author_id,title
0,1000,1000,636,999


**book_id** ожидаемо уникальное поле, **author_id** ожидаемо не уникальное (есть повторы), в **title** есть один повтор. 
Найдем его:

In [11]:
duplicate_check = 'select * from books where title in (SELECT title from books group by title having count(title)>1)'
pd.io.sql.read_sql(duplicate_check, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,426,39,Memoirs of a Geisha,434,2005-11-15,241
1,427,39,Memoirs of a Geisha,503,2005-11-22,311


У книги один автор, одно название, но разные годы публикации и разные издатели. Будем учитывать это при построении запросов.

In [21]:
#Посчитайте, сколько книг вышло после 1 января 2000 года;
books_count_query= '''SELECT COUNT(book_id)
                       FROM books 
                       WHERE publication_date > '2000-01-01'
                    '''
pd.io.sql.read_sql(books_count_query, con = engine)

,count
0,819


После 1 января 2000 года вышла 821 книга

In [12]:
#Для каждой книги посчитать количество обзоров и среднюю оценку;
count_avg = "SELECT b.title, COUNT(distinct(r.review_id)), AVG(rt.rating) FROM reviews r RIGHT JOIN books b ON r.book_id = b.book_id INNER JOIN ratings rt ON b.book_id = rt.book_id GROUP BY b.book_id ORDER BY count, avg"
pd.io.sql.read_sql(count_avg, con = engine)

,title,count,avg
0,The Natural Way to Draw,0,3.000000
1,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
2,Leonardo's Notebooks,0,4.000000
3,Essential Tales and Poems,0,4.000000
4,Disney's Beauty and the Beast (A Little Golden...,0,4.000000
...,...,...,...
995,The Glass Castle,6,4.206897
996,The Book Thief,6,4.264151
997,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
998,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634


Количество обзоров книг - от 0 до 7, а средний рейтинг книг - от 3.0 до 4.4

In [13]:
#Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
publisher_count = "SELECT p.publisher, COUNT(distinct(b.title)) FROM publishers p JOIN books b ON p.publisher_id=b.publisher_id WHERE num_pages > 50 GROUP BY p.publisher ORDER BY count desc LIMIT 1"
pd.io.sql.read_sql(publisher_count, con = engine)

,publisher,count
0,Penguin Books,42


Издательство, которое выпустило больше всего книг толще 50 страниц - Penguin Books, оно выпустило 42 книги

In [14]:
#Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
author_avg_rating ="SELECT a.author,COUNT(r.rating_id), AVG(r.rating) FROM authors a INNER JOIN books b ON b.author_id=a.author_id INNER JOIN ratings r ON r.book_id=b.book_id GROUP BY a.author HAVING COUNT(r.rating_id) > 50 ORDER BY AVG(r.rating) DESC LIMIT 1"
pd.io.sql.read_sql(author_avg_rating, con = engine)

,author,count,avg
0,J.K. Rowling/Mary GrandPré,312,4.288462


Автор J.K. Rowling/Mary GrandPr имеет самую высокую среднюю оценку книг - 4.288 среди книг с 50ю оценками и более

In [56]:
#Посчитайть среднее количество обзоров от пользователей, которые поставили больше 50 оценок.
o=''' SELECT AVG(b)
FROM (SELECT r.username, COUNT(distinct r.review_id) AS b
FROM (SELECT username, COUNT(DISTINCT rating_id)
    FROM ratings
    GROUP BY username
    HAVING COUNT(DISTINCT rating_id)>50) AS o
    JOIN reviews r ON r.username=o.username
    GROUP BY r.username) AS v
    '''
pd.io.sql.read_sql(o, con = engine)

,avg
0,24.333333


Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок ~ 24.33

# Вывод

В ходе исследования проанализировали базу данных, а также решили поставленные задачи и получили следующие результаты:

* После 1 января 2000 года вышла 821 книга
* Количество обзоров книг - от 0 до 7, а средний рейтинг книг - от 3.0 до 4.4
* Издательство, которое выпустило больше всего книг толще 50 страниц - Penguin Books, оно выпустило 42 книги
* Автор J.K. Rowling/Mary GrandPr имеет самую высокую среднюю оценку книг - 4.288 среди книг с 50ю оценками и более
* Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок ~ 24.33